In [50]:
import os
import json


def get_labels(audio_paths):
    labels = []
    with open('config.json', 'r') as config_file:
        config = json.load(config_file)
    
    for path in audio_paths:
        word = path.split('\\')[0]
        labels.append(config["label"][word])
    
    return labels


def get_path(mode):
    
    with open('data/testing_list.txt', 'r', encoding='utf-8') as f:
        test_path = f.readlines()
        test_path = [element.strip() for element in test_path if element]

    with open('data/validating_list.txt', 'r', encoding='utf-8') as f:
        val_path = f.readlines()
        val_path = [element.strip() for element in val_path if element]

    with open('data/training_list.txt', 'r', encoding='utf-8') as f:
        train_path = f.readlines()
        train_path = [element.strip() for element in train_path if element]

    # train_path = []
    # for root, dirs, files in os.walk('data/data_mini_merge'):
    #     for filename in files:
    #         if filename.endswith('.wav'):
    #             # 打印文件的完整路径
    #             path = os.path.join(root, filename)
    #             path = path.split('data_mini_merge\\')[-1]
    #             train_path.append(path)
    # with open('data/training_list.txt', 'w', encoding='utf-8') as f:
    #     f.write('\n'.join(train_path))
    if mode == 'test':
        return test_path
    elif mode == 'val':
        return val_path
    elif mode == 'train':
        return train_path


# path = get_path('train')
# label = get_labels(path)

In [43]:
import wave
import numpy as np

# 归一化函数
def normalize(wav_data):
    max_val = np.max(np.abs(wav_data))
    if max_val > 0:
        return wav_data / max_val
    return wav_data

# 长度标准化函数 - 这里我们选择填充（扩充）方式
def standardize_length(wav_data, desired_length):
    current_length = len(wav_data)
    if current_length == desired_length:
        return wav_data
    elif current_length < desired_length:
        padding = np.zeros(desired_length - current_length)
        return np.concatenate((wav_data, padding))
    else:  # 当前长度大于期望长度时进行截断
        return wav_data[:desired_length]

# 读取WAV文件并处理
def process_wav_files(wav_file_name, desired_length):

    # 打开WAV文件
    with wave.open(wav_file_name, 'rb') as wav_file:
        # 读取音频数据
        wav_data = np.frombuffer(wav_file.readframes(wav_file.getnframes()), dtype=np.int16)
        
        # 归一化音频数据
        wav_data_normalized = normalize(wav_data)
        
        # 长度标准化
        wav_data_standardized = standardize_length(wav_data_normalized, desired_length)

        return wav_data_standardized


In [44]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from wave import open as wave_open
import pandas as pd

class WavDataset(Dataset):
    def __init__(self, audio_paths, labels, config, transform=None, desired_length=16000):
        self.config = config
        self.audio_paths = audio_paths
        self.labels = pd.get_dummies(labels, dummy_na=False).reindex(columns=self.config["label"].values(), fill_value=False)
        self.labels = torch.tensor(np.array(self.labels.astype(int)))
        self.transform = transform
        self.desired_length = desired_length
        

    def __len__(self):
        return len(self.audio_paths)

    def __getitem__(self, idx):
        # 加载WAV文件
        file_path = os.path.join(self.config["data_path"], self.audio_paths[idx])
        wav_data = process_wav_files(file_path, self.desired_length)

        # 应用transform
        if self.transform:
            wav_data = self.transform(wav_data)

        # 标签
        label = self.labels[idx] if self.labels is not None else None

        return wav_data, label





In [49]:
# 假设你已经有了WAV文件路径列表和对应的标签列表
with open('config.json', 'r') as config_file:
    config = json.load(config_file)
path = get_path('train')
label = get_labels(path)

# 创建数据集实例
dataset = WavDataset(path, label, config)

dataloader = DataLoader(dataset, batch_size=2)

for batch in dataloader:
    print(batch[0].size())
    break

torch.Size([2, 16000])


In [46]:
a = [3, 8]
all_categories = config["label"].values()
pd.get_dummies(a, dummy_na=False).reindex(columns=config["label"].values(), fill_value=False)

,0,1,2,3,4,5,6,7,8,9,10,11
0,False,False,False,True,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,True,False,False,False
